<a href="https://colab.research.google.com/github/angyf/Proyecto_Eq_10/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Debido a que la API que complementa la información del proyecto no puede ser automatizada, se decidio trabajar con una diferente. Se escogio la API de superheroes con el fin de conocer mas sobre las caracteristicas generales de estos.

In [1]:
#importamos paqueterias utiles para la automatización de carga y limpieza
import requests
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np

In [2]:
#hacemos la carga de un dato con el fin de ver la información que es posible extraer
url='https://superheroapi.com/api/4258681504161074/1'
r=requests.get(url)
#veamos si la carga se realizo correctamente
r.status_code


200

In [3]:
#veamos el contenido de lla consulta
response=r.json()
response.keys()

dict_keys(['response', 'id', 'name', 'powerstats', 'biography', 'appearance', 'work', 'connections', 'image'])

Realizamos la carga de información sobre caracteristicas generales de los superheroes como lo son: nombre, superpoderes,editorial a la que pertenecen y caracteristicas físicas (peso y altura).

Al correr la siguiente celda se presenta una barra que presenta el progreso de a carga pues el tiempo que esto toma depende del nivel de conexión a internet que se tenga 

In [4]:
#automatizamos el proceso para la carga de 
df=pd.DataFrame()
for i in tqdm(range(1,732)):
  try:
    url=f'https://superheroapi.com/api/4258681504161074/{i}'
    r=requests.get(url)
    if r.status_code==200:
      response=r.json()
      datos_normalizados_1=pd.json_normalize(response['powerstats'])
      datos_normalizados_2=pd.Series(response['biography']['publisher'])
      datos_normalizados_3=pd.json_normalize(response['appearance'])
      name=pd.Series(response['name'])
      registro=pd.concat([name,datos_normalizados_1,datos_normalizados_2,datos_normalizados_3],axis=1)
      df=pd.concat([df,registro],axis=0)
      
  except:
    continue



In [5]:
#vamos nuestros datos
df.head()

,0,intelligence,strength,speed,durability,power,combat,1,gender,race,height,weight,eye-color,hair-color
0,A-Bomb,38,100,17,80,24,64,Marvel Comics,Male,Human,"[6'8, 203 cm]","[980 lb, 441 kg]",Yellow,No Hair
0,Abe Sapien,88,28,35,65,100,85,Dark Horse Comics,Male,Icthyo Sapien,"[6'3, 191 cm]","[145 lb, 65 kg]",Blue,No Hair
0,Abin Sur,50,90,53,64,99,65,DC Comics,Male,Ungaran,"[6'1, 185 cm]","[200 lb, 90 kg]",Blue,No Hair
0,Abomination,63,80,53,90,62,95,Marvel Comics,Male,Human / Radiation,"[6'8, 203 cm]","[980 lb, 441 kg]",Green,No Hair
0,Abraxas,88,63,83,100,100,55,Marvel Comics,Male,Cosmic Entity,"[-, 0 cm]","[- lb, 0 kg]",Blue,Black


# <center> Limpieza de la base de datos <center>

* Tenemos que reindexar la información pues todos los registros presentan un indice igual a cero. 
 
* Tambien notamos que podríamos presentar de mejor forma el peso y la altura con un solo registro en estas columnas.

* Así mismo hay nombres de columnas que no nos dicen nada por lo que es mejor cambiar los nombres.

* Finalmente revisaremos que el tipo de dato corresponda a lo que se presenta, en caso de que esto no suceda se realizaran las modificaciones necesarias


In [6]:
#reindexamos el DF
df=df.reset_index(drop=True)


In [7]:
#obtenemos el peso en kg, en caso de no haber registro se registra un NaN
def repl(x):
  try:
    x=str(x[1])
    x=str(x.replace('kg','')).strip()
    x=int(x)
  except:
    x=float('NaN')
  return x
df['weight']=df['weight'].apply(repl)

In [8]:
#Obtenemos la altura en cm y la convertimos a metros
# en caso de no haber registro se registra una NaN
def repl2(x):
  try:
    x=str(x[1])
    x=str(x.replace('cm','')).strip()
    x=int(x)
    x/=100
  except:
    x=float('NaN')
  return x
df['height']=df['height'].apply(repl2)


In [9]:
#renombraremos algunas columnas con nombres mas explicitos
nombres={
    0:'name',
    1:'publisher',
    'height':'height_m',
    'weight':'weight_kg'
}

df=df.rename(columns=nombres)


In [10]:
df.dtypes

name             object
intelligence     object
strength         object
speed            object
durability       object
power            object
combat           object
publisher        object
gender           object
race             object
height_m        float64
weight_kg       float64
eye-color        object
hair-color       object
dtype: object

Notamos que los tipos de datos no son los correctos, por lo que realizaremos las transformación

In [11]:
#reliazamos una copia para que no se modifiquen los datos extraidos originalmente
df1= df.copy()
#Lista auxiliar
columnas=['intelligence','strength','speed','durability','power','combat']
#eliminamos columnas que no sirven para el análisis
del df1['race']
#modificamos las columnas que presentan datos numericos       
for columna in columnas:
  df1[columna]=pd.to_numeric(df1[columna],errors='coerce')
#Revisamos de nuevo el tipo de datos de nuestra base
df1.dtypes

name             object
intelligence    float64
strength        float64
speed           float64
durability      float64
power           float64
combat          float64
publisher        object
gender           object
height_m        float64
weight_kg       float64
eye-color        object
hair-color       object
dtype: object

Limpieza de datos nulos

In [12]:
#veamos cuantos nulos tenemos
df1.isna().sum()

name              0
intelligence    165
strength        102
speed           165
durability      165
power           165
combat          165
publisher         0
gender            0
height_m          8
weight_kg         7
eye-color         0
hair-color        0
dtype: int64

In [13]:
#Quitaremos los registros con datos NaN
df1=df1.dropna(axis=0, how='any')
#Debido a que el peso y la altura no puden ser 0 o negativos se quitaran tambien estos registros 
filtro_peso=df1['weight_kg']>0
filtro_altura=df1['height_m']>0



Finalmente tenemos una base de datos limpia para un análisis.

In [14]:
#aplicamos un filtro para restringir que peso y altura sean no negativos
df_clean=df1[filtro_peso & filtro_altura]

#creamos la base limpia que servira para analisis.
df_clean=df_clean.reset_index(drop=True)

df_clean

,name,intelligence,strength,speed,durability,power,combat,publisher,gender,height_m,weight_kg,eye-color,hair-color
0,A-Bomb,38.0,100.0,17.0,80.0,24.0,64.0,Marvel Comics,Male,2.03,441.0,Yellow,No Hair
1,Abe Sapien,88.0,28.0,35.0,65.0,100.0,85.0,Dark Horse Comics,Male,1.91,65.0,Blue,No Hair
2,Abin Sur,50.0,90.0,53.0,64.0,99.0,65.0,DC Comics,Male,1.85,90.0,Blue,No Hair
3,Abomination,63.0,80.0,53.0,90.0,62.0,95.0,Marvel Comics,Male,2.03,441.0,Green,No Hair
4,Absorbing Man,38.0,80.0,25.0,100.0,98.0,64.0,Marvel Comics,Male,1.93,122.0,Blue,No Hair
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,Yellowjacket,88.0,10.0,12.0,28.0,12.0,14.0,Ant-Man,Male,1.83,83.0,Blue,Blond
424,Yellowjacket II,50.0,10.0,35.0,28.0,31.0,28.0,Marvel Comics,Female,1.65,52.0,Blue,Strawberry Blond
425,Yoda,88.0,52.0,33.0,25.0,100.0,90.0,George Lucas,Male,0.66,17.0,Brown,White
426,Zatanna,81.0,10.0,23.0,28.0,100.0,56.0,DC Comics,Female,1.70,57.0,Blue,Black


Algunos análisis y filtros


In [15]:
#descripción general de las columnas
df_clean.describe()

,intelligence,strength,speed,durability,power,combat,height_m,weight_kg
count,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000
mean,64.813084,42.567757,40.672897,60.112150,64.824766,64.000000,1.843061,115.525701
std,19.066201,32.575297,23.426411,28.591634,29.012226,22.224425,0.253003,107.305215
min,6.000000,5.000000,1.000000,1.000000,0.000000,5.000000,0.640000,14.000000
25%,50.000000,10.000000,23.000000,32.000000,40.000000,50.000000,1.730000,61.000000
50%,63.000000,32.000000,35.000000,60.000000,64.500000,64.000000,1.830000,82.500000
75%,75.000000,73.500000,50.000000,85.000000,100.000000,80.000000,1.910000,108.000000
max,100.000000,100.000000,100.000000,100.000000,100.000000,101.000000,3.660000,900.000000


In [16]:
# Top 5 de mujeres más poderosas de Marvel
marvel=df_clean[df_clean['publisher']=='Marvel Comics']
mujeres_marvel=marvel[marvel['gender']=='Female']
(mujeres_marvel.sort_values('power',ascending=False).head(5))[['name','power']]

,name,power
238,Mantis,100.0
114,Dazzler,100.0
342,Sif,100.0
28,Ardina,100.0
330,Scarlet Witch,100.0


In [17]:
# Top 5 de hombres más poderosos de Marvel
hombres_marvel=marvel[marvel['gender']=='Male']
(hombres_marvel.sort_values('power',ascending=False).head())[['name','power']]

,name,power
422,X-Man,100.0
396,Ultron,100.0
343,Silver Surfer,100.0
245,Mephisto,100.0
158,Ghost Rider,100.0
